In [ ]:
import pandas as pd
import sqlite3
import os


In [ ]:
# Define the path for your SQLite database file
db_path = '/kaggle/working/music_database.db'

# Connect to the SQLite database (it will create the file if it doesn't exist)
conn = sqlite3.connect(db_path)


In [ ]:
# Path to the folder containing your CSV files
csv_folder_path = '/kaggle/input/music-store-data-csvs/'

# Loop through all CSV files in the folder
for csv_file in os.listdir(csv_folder_path):
    if csv_file.endswith('.csv'):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(csv_folder_path, csv_file))
        
        # Use the filename (without extension) as the table name
        table_name = os.path.splitext(csv_file)[0]
        
        # Write the DataFrame to the SQLite database
        df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection to the database
conn.close()


In [ ]:
# Reconnect to the database
conn = sqlite3.connect(db_path)

# Query to get the list of tables
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# Print the list of tables
print("Tables in SQLite database:")
for table in tables:
    print(table[0])

# Close the connection
conn.close()

In [ ]:
import shutil

# Move the SQLite file to the output directory
shutil.move(db_path, '/kaggle/working/my_database.db')

In [ ]:
%load_ext sql

In [ ]:
# Set the connection to the SQLite database
%sql sqlite:////kaggle/working/my_database.db

In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';


## 1. Who is the senior most employee based on job title

In [ ]:
%%sql
SELECT * FROM employee
ORDER BY levels DESC
LIMIT 1;